In [1]:
import os

MODEL = "gemma2"


In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings  import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

chain.invoke("Tell me a joke")  

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("aiesec.pdf")
pages = loader.load_and_split()
pages

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")


In [ ]:
chain = prompt |  model | parser

In [ ]:
chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

In [ ]:
chain.input_schema.schema()

In [ ]:
chain.invoke(
    {
        "context": "My parents named me Santiago", 
        "question": "What's your name'?"
     }

)

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, 
                                                    embedding=embeddings
                                                    )



In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("Exchange Standards")


In [ ]:
from operator import itemgetter

chain = (
{
    'context' : itemgetter('question') | retriever,
    'question' : itemgetter('question')
}
prompt | model | parser
)
chain.invoke({"question": "What is the exchange standards?"})